# PyLCM

In [27]:
#import necessary modules
from PyLCM.animation import *
from PyLCM.widget import *
from PyLCM.timestep_routine import *
#import user modifiable modules
from Post_process.analysis import *
from Post_process.print_plot import *

# uncomment lines below when changing source files
#######################
%load_ext autoreload
%autoreload 2
#######################

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Intialization
## Input parameter

### Model steering parameters

In [28]:
dt_widget, nt_widget, Condensation_widget, Collision_widget, n_particles_widget, max_z_widget = model_steering_input()

'Model steering parameters'

BoundedFloatText(value=0.5, description='dt [s]:', max=5.0, min=0.0001)

IntText(value=3600, description='nt:')

Checkbox(value=True, description='Condensation:')

Checkbox(value=False, description='Collision:')

BoundedIntText(value=500, description='n_particles:', max=1000, min=100)

BoundedFloatText(value=1400.0, description='z_max [m]:', max=1400.0, step=0.1)

### Parcel parameters

In [29]:
T_widget, P_widget, RH_widget, w_widget = parcel_info_input()

ascending_mode_widget = ascending_mode_input()

'Parcel initial parameters: '

BoundedFloatText(value=293.2, description='T [K]:', max=320.0, min=200.0, step=0.1)

BoundedFloatText(value=101300.0, description='P [Pa]:', max=105000.0, min=95000.0, step=1.0)

BoundedFloatText(value=0.88, description='RH [-]:', max=0.99, min=0.01, step=0.01)

BoundedFloatText(value=0.5, description='w [m/s]:', max=10.0, step=0.1)

ToggleButtons(description='Mode', layout=Layout(width='max-content'), options=('linear', 'sine', 'in_cloud_osc…

### Aerosol parameters

In [83]:
# choose the mode for aerosol initialization
mode_aero_init_widget = aero_mode_input()
# Initialization parameters for aerosol (up to 4 modes)
gridwidget = grid_modes_input()

#use activation radius with kohler critical radius
kohler_activation_radius = False
#use fixed activation radius for aerosol/droplet seperation
if not kohler_activation_radius:
    act_crit_r = 1.0E-6

'Aerosol initialisation mode: '

ToggleButtons(layout=Layout(width='max-content'), options=('weighting_factor', 'random'), value='weighting_fac…

N_aero: number of aerosols per cubic centimeter, mu: mean droplet radius, sigma: std of mu


GridspecLayout(children=(Button(button_style='info', description='Mode #1 (plot)', layout=Layout(grid_area='wi…

# Time step routine

In [84]:
# setting of display mode while running: either output of variables in text form (fast, recommended) = 'text'
# or: continuously updated plot using plotly (slow, time & memory consuming) = 'graphics'
mode_displaytype_widget = timestep_display_mode_settings()

ToggleButtons(description='display mode:', layout=Layout(width='max-content'), options=('text_fast', 'graphics…

In [ ]:
# initialization of the model (in aero_init module)
mode_aero_init, n_particles, P_parcel, T_parcel, q_parcel, z_parcel, w_parcel, \
N_aero, mu_aero, sigma_aero, nt, dt, max_z, do_condensation, \
do_collision = model_init(dt_widget, nt_widget, Condensation_widget, Collision_widget, \
                                                n_particles_widget, T_widget, P_widget, RH_widget, \
                                                w_widget, max_z_widget, mode_aero_init_widget, gridwidget)

# time step routine (in model timestep_routine)
time_array, T_parcel_array, RH_parcel_array, q_parcel_array, z_parcel_array, \
qa_ts,qc_ts,qr_ts,na_ts,nc_ts,nr_ts, spectra_arr = timesteps_function(mode_aero_init, n_particles, P_parcel, T_parcel, q_parcel, z_parcel, w_parcel, N_aero, mu_aero,\
                                                                       sigma_aero, rho_aero, molecular_weight_aero, nt, dt, rm_spec, \
                                                                       ascending_mode_widget, mode_displaytype_widget, max_z, \
                                                                       do_condensation, do_collision, kohler_activation_radius, act_crit_r)

value: Time (s)  z (m)    T (K)    qv (g/kg) RH (%)   QC (g/kg) QR (g/kg) NA (/mg) NC (/mg) NR (/mg)
after: 6.0       3.00     293.17   12.95     88.159    0.000     0.000     1977.24   0.07      0.00    


# post processing and analysis
## plots
plots can be drawn either as time-series or as vertical profiles

In [22]:
# user can choice if he wants the plots time-dependent or height-dependent (except of DSD and particle densities always being time dependent)
mode_plots_widget = plot_widgets_settings()

ToggleButtons(description='plots are:', layout=Layout(width='max-content'), options=('time-series', 'vertical …

In [23]:
plot_mode=mode_plots_widget.value
subplot_array_function(plot_mode, dt, nt,rm_spec, qa_ts, qc_ts, qr_ts, na_ts, nc_ts, nr_ts, T_parcel_array, RH_parcel_array, q_parcel_array, z_parcel_array, spectra_arr)

NameError: name 'qa_ts' is not defined

## data output
options for saving output variables and drop size spectra, files will be written in the subfolder 'output'

In [24]:
# save array of output variables into csv-file, optional filename can be given as filename='my_filename.csv'
save_model_output_variables(time_array, RH_parcel_array, q_parcel_array, T_parcel_array, z_parcel_array, qa_ts, \
                            qc_ts, qr_ts, na_ts, nc_ts, nr_ts, filename='testoutput_model_230809.csv')

NameError: name 'time_array' is not defined

In [13]:
# save array of DSD spectra per timesteps, combined with radii to csv-file
save_model_output_dsd(spectra_arr, rm_spec, rl_spec, rr_spec, nt, filename='dsd_array_output_test_230822.csv')

Output data of droplet size distribution written to: output/dsd_array_output_test_230822.csv
